# Day 2A: Agent Tools

## 🎯 Learning Objectives

By the end of this notebook, you will:
- Understand what tools are in the context of AI agents
- Learn how to create custom function tools from Python code
- Master using agents as tools for specialized tasks
- Improve agent reliability with code execution
- Know when to use different types of tools

## 📚 Prerequisites

- Completion of **Day 1A: Introduction to Agent Architectures**
- Completion of **Day 1B: Advanced Agent Architectures** (recommended)
- Understanding of basic agent components
- Google API key (stored in `.env` file)

---


## What Are Agent Tools?

Think of tools as **actions that agents can take**. Just like humans use tools (hammers, computers, phones), AI agents use tools to interact with the world beyond just generating text.

### Tool Analogy

```

                Human Assistant:                     AI Agent:
                ┌─────────────────┐                 ┌─────────────────┐
                │  "What's the    │                 │  "What's the    │
                │   weather?"     │                 │   weather?"     │
                └───────┬─────────┘                 └───────┬─────────┘
                        │                                   │
                        ▼                                   ▼
                ┌─────────────────┐                 ┌─────────────────┐
                │ Opens weather   │                 │ Uses google_    │
                │ app on phone    │                 │ search tool     │
                └───────┬─────────┘                 └─────────────────┘
                        │                                   │
                        ▼                                   ▼
                ┌─────────────────┐                 ┌─────────────────┐
                │ "It's 15°C and  │                 │ "It's 15°C and  │
                │  cloudy"        │                 │  cloudy"        │
                └─────────────────┘                 └─────────────────┘

```

### Why Do Agents Need Tools?

Without tools, agents can only:
- Answer based on their training data
- Generate text responses
- Be limited to knowledge cutoff date

With tools, agents can:
- Search for current information
- Perform calculations accurately
- Access external APIs and databases
- Execute code and process files
- Take real-world actions

---


## The ADK Tool Ecosystem

ADK provides a comprehensive toolkit organized into two main categories:

```
        ┌─────────────────────────────────────────────────────────┐
        │                    ADK Tool Ecosystem                   │
        │                                                         │
        │  ┌─────────────────┐    ┌─────────────────────────────┐ │
        │  │   Custom Tools  │    │      Built-in Tools         │ │
        │  │                 │    │                             │ │
        │  │ • Function      │    │ • Gemini Tools              │ │
        │  │ • Agent         │    │   - Google Search           │ │
        │  │ • Long Running  │    │   - Code Execution          │ │
        │  │ • MCP           │    │                             │ │
        │  │ • OpenAPI       │    │ • Google Cloud Tools        │ │
        │  │                 │    │   - BigQuery                │ │
        │  └─────────────────┘    │   - Spanner                 │ │
        │                         │                             │ │
        │                         │ • Third-party Tools         │ │
        │                         │   - Hugging Face            │ │
        │                         │   - GitHub                  │ │
        │                         └─────────────────────────────┘ │
        └─────────────────────────────────────────────────────────┘

```

### Tool Selection Guide

| Tool Type | When to Use | Example | Complexity |
|-----------|-------------|---------|-------------|
| **Function Tool** | Custom logic, calculations | Currency converter | Low |
| **Agent Tool** | Specialized expertise | Math specialist agent | Medium |
| **MCP Tool** | External services | Filesystem access | Medium |
| **Gemini Tool** | Common needs | Google Search | Low |

---



## Setting Up Our Environment

Let's install the required packages:

In [ ]:
import sys
import subprocess
import importlib

def install_package(package):
    """Install a Python package using pip"""
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Check if python-dotenv is installed, install if not
try:
    import dotenv
    print("✅ python-dotenv is already installed")
except ImportError:
    print("Installing python-dotenv...")
    install_package("python-dotenv")
    import dotenv
    print("✅ python-dotenv installed successfully")

# Check if google-adk is installed, install if not
try:
    import google.adk.agents
    print("✅ google-adk is already installed")
except ImportError:
    print("Installing google-adk...")
    install_package("google-adk")
    import google.adk.agents
    print("✅ google-adk installed successfully")

# Check if google-genai is installed, install if not
try:
    import google.genai
    print("✅ google-genai is already installed")
except ImportError:
    print("Installing google-genai...")
    install_package("google-genai")
    import google.genai
    print("✅ google-genai installed successfully")

print("\n🎉 All required packages are installed and ready!")

## Setting Up Authentication

Load the API key from your `.env` file:

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

try:
    # Get API key from environment variable (from .env file)
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    
    if not GOOGLE_API_KEY:
        raise ValueError("GOOGLE_API_KEY not found in environment variables")
    
    # Set the API key for ADK
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    
    print("✅ Gemini API key setup complete.")
    
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have set 'GOOGLE_API_KEY' in your .env file. Details: {e}")

## Importing ADK Components

For working with tools, we need additional components:

In [ ]:
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")
print("📦 Available components:")
print("  • LlmAgent - Advanced agent class with tool support")
print("  • Gemini - Model configuration with retry options")
print("  • AgentTool - Use agents as tools")
print("  • BuiltInCodeExecutor - Execute Python code")
print("  • ToolContext - Access tool execution context")

## Configuring Retry Options

When working with external services and LLMs, you may encounter transient errors. Let's configure retry options to make our agents more reliable:

In [ ]:
# Configure retry options for handling transient errors
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier (exponential backoff)
    initial_delay=1,  # Initial delay in seconds
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

print("✅ Retry configuration created.")
print(f"🔄 Will retry up to {retry_config.attempts} times for errors: {retry_config.http_status_codes}")

---

## Section 1: Building Custom Function Tools

Function tools are the most common way to extend agent capabilities. Any Python function can become an agent tool!

### Example: Currency Converter Agent

Let's build a practical agent that can convert currency and calculate fees. This agent will:

1. **Fee Lookup Tool** - Find transaction fees for payment methods
2. **Exchange Rate Tool** - Get current currency conversion rates
3. **Calculation Step** - Calculate total conversion cost including fees

### How to Create a Function Tool

Creating a function tool is simple:

```
1. Write a Python function
2. Follow ADK best practices (see below)
3. Add it to agent's tools list
4. ADK handles the rest!
```

### ADK Best Practices for Function Tools

✅ **DO:**
- Return structured dictionaries with status field
- Write clear, descriptive docstrings
- Use type hints for parameters and return values
- Handle errors gracefully
- Keep functions focused on one task

❌ **DON'T:**
- Return raw values without status information
- Write vague or missing docstrings
- Let exceptions propagate without handling
- Create functions that do too many things

### Creating the Fee Lookup Tool

In [ ]:
def get_fee_for_payment_method(method: str) -> dict:
    """Looks up the transaction fee percentage for a given payment method.

    This tool simulates looking up a company's internal fee structure based on the
    name of the payment method provided by the user.

    Args:
        method: The name of the payment method. It should be descriptive,
                e.g., "platinum credit card" or "bank transfer".

    Returns:
        Dictionary with status and fee information.
        Success: {"status": "success", "fee_percentage": 0.02}
        Error: {"status": "error", "error_message": "Payment method not found"}
    """
    # This simulates looking up a company's internal fee structure.
    fee_database = {
        "platinum credit card": 0.02,  # 2%
        "gold debit card": 0.035,  # 3.5%
        "bank transfer": 0.01,  # 1%
        "paypal": 0.025,  # 2.5%
        "wire transfer": 0.015,  # 1.5%
    }

    fee = fee_database.get(method.lower())
    if fee is not None:
        return {"status": "success", "fee_percentage": fee}
    else:
        return {
            "status": "error",
            "error_message": f"Payment method '{method}' not found. Available methods: {list(fee_database.keys())}",
        }

print("✅ Fee lookup function created")
print(f"💳 Test: {get_fee_for_payment_method('platinum credit card')}")
print(f"💳 Test: {get_fee_for_payment_method('unknown method')}")

### Creating the Exchange Rate Tool

In [ ]:
def get_exchange_rate(base_currency: str, target_currency: str) -> dict:
    """Looks up and returns the exchange rate between two currencies.

    Args:
        base_currency: The ISO 4217 currency code of the currency you
                       are converting from (e.g., "USD").
        target_currency: The ISO 4217 currency code of the currency you
                         are converting to (e.g., "EUR").

    Returns:
        Dictionary with status and rate information.
        Success: {"status": "success", "rate": 0.93}
        Error: {"status": "error", "error_message": "Unsupported currency pair"}
    """
    # Static data simulating a live exchange rate API
    # In production, this would call something like: requests.get("api.exchangerates.com")
    rate_database = {
        "usd": {
            "eur": 0.93,  # Euro
            "jpy": 157.50,  # Japanese Yen
            "inr": 83.58,  # Indian Rupee
            "gbp": 0.79,  # British Pound
            "cad": 1.36,  # Canadian Dollar
        },
        "eur": {
            "usd": 1.08,  # US Dollar
            "gbp": 0.85,  # British Pound
            "jpy": 169.89,  # Japanese Yen
        }
    }

    # Input validation and processing
    base = base_currency.lower()
    target = target_currency.lower()

    # Return structured result with status
    rate = rate_database.get(base, {}).get(target)
    if rate is not None:
        return {"status": "success", "rate": rate}
    else:
        return {
            "status": "error",
            "error_message": f"Unsupported currency pair: {base_currency.upper()}/{target_currency.upper()}",
        }

print("✅ Exchange rate function created")
print(f"💱 Test: {get_exchange_rate('USD', 'EUR')}")
print(f"💱 Test: {get_exchange_rate('EUR', 'USD')}")
print(f"💱 Test: {get_exchange_rate('USD', 'XYZ')}")

### Creating the Currency Agent

Now let's create an agent that uses these tools. Note how the agent's instructions reference the tools by name:

In [ ]:
# Currency agent with custom function tools
currency_agent = LlmAgent(
    name="currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a smart currency conversion assistant.

For currency conversion requests:
1. Use `get_fee_for_payment_method()` to find transaction fees
2. Use `get_exchange_rate()` to get currency conversion rates
3. Check the "status" field in each tool's response for errors
4. Calculate the final amount after fees based on the output from the tools
5. Provide a clear breakdown showing:
   - The final converted amount
   - The fee percentage and its value in the original currency
   - The amount remaining after the fee
   - The exchange rate used for the final conversion

If any tool returns status "error", explain the issue to the user clearly.
""",
    tools=[get_fee_for_payment_method, get_exchange_rate],
)

print("✅ Currency agent created with custom function tools")
print("🔧 Available tools:")
print("  • get_fee_for_payment_method - Looks up company fee structure")
print("  • get_exchange_rate - Gets current exchange rates")
print(f"🤖 Agent model: {currency_agent.model.model_id}")

### Agent vs. LlmAgent: What's the Difference?

You might notice we're using `LlmAgent` here while Day1a/Day1b used the `Agent` class. Here's the key difference:

| Feature | Agent (Day1a/Day1b) | LlmAgent (Day2a) |
|---------|----------------------|-------------------|
| **Model Specification** | String: `"gemini-2.5-flash-lite"` | Object: `Gemini(model="gemini-2.5-flash-lite", retry_options=...)` |
| **Configuration** | Basic | Advanced (retry options, timeouts, etc.) |
| **Code Execution** | Not directly supported | Built-in support via `code_executor` parameter |
| **Use Case** | Simple agents | Complex agents with custom tools and advanced features |

**Why LlmAgent here?** Day2a focuses on custom tools and code execution, which requires advanced configuration options that `LlmAgent` provides.

### Testing the Currency Agent

In [ ]:
# Test the currency agent
currency_runner = InMemoryRunner(agent=currency_agent)

print("💱 Testing currency agent...")
print("💵 Query: I want to convert 500 US Dollars to Euros using my Platinum Credit Card. How much will I receive?")
print("\n⏳ Processing...")

response = await currency_runner.run_debug(
    "I want to convert 500 US Dollars to Euros using my Platinum Credit Card. How much will I receive?"
)

print("\n💰 Conversion Result:")
print(response)

---

## Section 2: Improving Reliability with Code Execution

The agent's instruction says "calculate the final amount after fees" but LLMs aren't always reliable at math. They might make calculation errors or use inconsistent formulas.

### The Problem: LLM Math Reliability

```
LLM Mental Math:
❌ Prone to errors
❌ Inconsistent formulas
❌ No verification
❌ Hard to debug

Code Execution:
✅ Precise calculations
✅ Consistent logic
✅ Verifiable results
✅ Easy to debug
```

### The Solution: Delegate Calculations to Code

Instead of asking the LLM to do math, we'll:
1. Ask the agent to generate Python code for the calculation
2. Execute that code using ADK's BuiltInCodeExecutor
3. Return the precise result

### Creating a Calculation Specialist Agent

In [ ]:
# Helper function to display code execution results
def show_python_code_and_result(response):
    """Helper function that prints the generated Python code and results from the code execution tool."""
    for i in range(len(response)):
        # Check if the response contains a valid function call result from the code executor
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "🐍 Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("📊 Generated Python Response >> ", response_code["result"])

print("✅ Helper function defined for displaying code execution results")

# Create a calculation agent that generates and executes Python code
calculation_agent = LlmAgent(
    name="CalculationAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a specialized calculator that ONLY responds with Python code.

Your task is to take a calculation request and translate it into a single block of Python code that calculates the answer.
      
**RULES:**
1. Your output MUST be ONLY a Python code block.
2. Do NOT write any text before or after the code block.
3. The Python code MUST calculate the result.
4. The Python code MUST print the final result to stdout.
5. You are PROHIBITED from performing the calculation yourself.
   
Failure to follow these rules will result in an error.
       """,
    code_executor=BuiltInCodeExecutor(),  # Use the built-in Code Executor Tool
)

print("✅ Calculation agent created with code execution capabilities")
print("🔧 This agent will generate Python code and execute it reliably")

### Creating an Enhanced Currency Agent

Now let's update our currency agent to use the calculation specialist:

In [ ]:
# Enhanced currency agent that delegates calculations to a specialist
enhanced_currency_agent = LlmAgent(
    name="enhanced_currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.

For any currency conversion request:
1. Get Transaction Fee: Use the get_fee_for_payment_method() tool to determine the transaction fee.
2. Get Exchange Rate: Use the get_exchange_rate() tool to get the currency conversion rate.
3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
4. Calculate Final Amount (CRITICAL): You are strictly prohibited from performing any arithmetic calculations yourself.
   You must use the calculation_agent tool to generate Python code that calculates the final converted amount.
   This code will use the fee information from step 1 and the exchange rate from step 2.
5. Provide Detailed Breakdown: In your summary, you must:
   * State the final converted amount.
   * Explain how the result was calculated, including:
       - The fee percentage and the fee amount in the original currency.
       - The amount remaining after deducting the fee.
       - The exchange rate applied.
""",
    tools=[
        get_fee_for_payment_method,
        get_exchange_rate,
        AgentTool(agent=calculation_agent),  # Using another agent as a tool!
    ],
)

print("✅ Enhanced currency agent created")
print("🎯 New capability: Delegates calculations to specialist agent")
print("🔧 Tool types used:")
print("  • Function Tools (fees, rates)")
print("  • Agent Tool (calculation specialist)")

### Testing the Enhanced Agent

In [ ]:
# Define a runner for the enhanced agent
enhanced_runner = InMemoryRunner(agent=enhanced_currency_agent)

print("💱 Testing enhanced currency agent...")
print("💵 Query: Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation.")
print("\n⏳ Processing with reliable calculations...")

# Test the enhanced agent
response = await enhanced_runner.run_debug(
    "Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation."
)

print("\n💰 Enhanced Conversion Result:")
print(response)

print("\n🐍 Generated Python Code:")
show_python_code_and_result(response)

### Agent Tools vs Sub-Agents: What's the Difference?

This is a common question! Both involve using multiple agents, but they work very differently:

#### Agent Tools (what we're using)
```
Agent A calls Agent B as a tool
                ┌─────────────┐    calls ┌─────────────┐
                │   Agent A   │ ────────►│   Agent B   │
                │ (Coordinator│          │ (Specialist)│
                │   stays     │          │             │
                │  in control)│          └─────────────┘
                └─────────────┘                 │               
                        Agent A waits for       |
                        response from           │ response goes
                        Agent B                 │ back to A
                                                ▼
                                        ┌─────────────┐
                                        │   Agent A   │
                                        │ continues   │
                                        │   working   │
                                        ┴─────────────┘

```

- Agent A calls Agent B as a tool
- Agent B's response goes back to Agent A
- Agent A stays in control and continues
- **Use case**: Delegation for specific tasks

#### Sub-Agents (different pattern)
```
Agent A transfers control completely to Agent B
        ┌─────────────┐    handoff┌─────────────┐
        │   Agent A   │ ────────► │   Agent B   │
        │ (Tier 1)    │           │ (Tier 2)    │
        │             │           │             │
        │   gives up  │           │   takes     │
        │   control   │           │   over      │
        └─────────────┘           └─────────────┘
                                        │
                                        ▼
                                ┌─────────────┐
                                │   Agent B   │
                                │ handles all │
                                │ future input│
                                ┴─────────────┘
```

- Agent A transfers control completely to Agent B
- Agent B takes over and handles all future input
- Agent A is out of the loop
- **Use case**: Handoff to specialists (like customer support tiers)

**In our currency example**: We want the currency agent to get calculation results and continue working, so we use **Agent Tools**.

---

## Section 3: Complete Guide to ADK Tool Types

Now that you've seen tools in action, let's explore the complete ADK toolkit. Understanding all available tools helps you choose the right one for your needs.

### Custom Tools

Custom tools are tools you build yourself for specific needs.

#### Function Tools ✅ (You've used these!)

**What**: Python functions converted to agent tools

**Examples**: `get_fee_for_payment_method`, `get_exchange_rate`

**Advantage**: Turn any Python function into an agent tool instantly

**When to use**:
- Business logic specific to your application
- Calculations and data processing
- Integration with internal systems

#### Long Running Function Tools

**What**: Functions for operations that take significant time

**Examples**: Human-in-the-loop approvals, file processing

**Advantage**: Agents can start tasks and continue with other work while waiting

**When to use**:
- Tasks that take more than a few seconds
- Operations requiring human approval
- Batch processing jobs

#### Agent Tools ✅ (You've used these!)

**What**: Other agents used as tools

**Examples**: `AgentTool(agent=calculation_agent)`

**Advantage**: Build specialist agents and reuse them across different systems

**When to use**:
- Delegating specialized tasks
- Creating reusable expertise
- Separating concerns

#### MCP Tools

**What**: Tools from Model Context Protocol servers

**Examples**: Filesystem access, Google Maps, databases

**Advantage**: Connect to any MCP-compatible service without custom integration

**When to use**:
- Accessing external services
- Working with files and directories
- Integrating with third-party APIs

#### OpenAPI Tools

**What**: Tools automatically generated from API specifications

**Examples**: REST API endpoints become callable tools

**Advantage**: No manual coding — just provide an API spec and get working tools

**When to use**:
- Integrating with REST APIs
- Rapid prototyping with external services
- Standardized API interactions

### Built-in Tools

Built-in tools are pre-built tools provided by ADK.

#### Gemini Tools ✅ (You've used these!)

**What**: Tools that leverage Gemini's capabilities

**Examples**: `google_search`, `BuiltInCodeExecutor`

**Advantage**: Reliable, tested tools that work out of the box

**When to use**:
- Web search for current information
- Code execution for calculations
- Common agent needs

#### Google Cloud Tools [needs Google Cloud access]

**What**: Tools for Google Cloud services and enterprise integration

**Examples**: `BigQueryToolset`, `SpannerToolset`, `APIHubToolset`

**Advantage**: Enterprise-grade database and API access with built-in security

**When to use**:
- Enterprise applications
- Large-scale data processing
- Google Cloud integration

#### Third-party Tools

**What**: Wrappers for existing tool ecosystems

**Examples**: Hugging Face, Firecrawl, GitHub Tools

**Advantage**: Reuse existing tool investments — no need to rebuild what already exists

**When to use**:
- Leveraging existing platforms
- AI/ML model integration
- Web scraping and data extraction

---

## Section 4: Best Practices for Tool Development

Building reliable tools requires following certain patterns and practices. Here's what we've learned throughout this notebook:

### 1. Structured Return Values

Always return structured dictionaries with a status field:

```python
# ✅ Good - Structured with status
return {
    "status": "success",
    "data": result
}

# ✅ Good - Structured error
return {
    "status": "error",
    "error_message": "Description of what went wrong"
}

# ❌ Bad - Raw return without status
return result
```

### 2. Clear Documentation

Write comprehensive docstrings that explain:
- What the tool does
- What each parameter means
- What the return value looks like
- Examples of success and error cases

### 3. Type Hints

Use type hints for better code clarity and ADK integration:

```python
def my_tool(param1: str, param2: int) -> dict:
    """Tool description"""
    pass
```

### 4. Error Handling

Handle errors gracefully and provide meaningful error messages:

```python
try:
    # Try to do something
    result = risky_operation()
    return {"status": "success", "data": result}
except SpecificException as e:
    return {
        "status": "error",
        "error_message": f"Failed to do operation: {str(e)}"
    }
```

### 5. Single Responsibility

Each tool should do one thing well:

```python
# ✅ Good - Single responsibility
def get_user_by_id(user_id: str) -> dict:
    """Get user information by ID"""
    pass

def update_user_email(user_id: str, email: str) -> dict:
    """Update user email"""
    pass

# ❌ Bad - Multiple responsibilities
def get_or_update_user(user_id: str, email: str = None) -> dict:
    """Get user or update email if provided"""
    pass
```

### 6. Input Validation

Validate inputs before processing:

```python
def process_amount(amount: float) -> dict:
    """Process monetary amount"""
    if amount <= 0:
        return {
            "status": "error",
            "error_message": "Amount must be positive"
        }
    # Continue processing
```

### 7. Consistent Naming

Use clear, descriptive names for tools and parameters:

```python
# ✅ Good - Clear names
def calculate_compound_interest(principal: float, rate: float, periods: int) -> dict:
    pass

# ❌ Bad - Vague names
def calc(p: float, r: float, n: int) -> dict:
    pass
```

---
## 📚 Additional Resources

- [ADK Tools Documentation](https://google.github.io/adk-docs/tools/)
- [Function Tool Best Practices](https://google.github.io/adk-docs/best-practices/function-tools/)
- [Code Execution Guide](https://google.github.io/adk-docs/code-execution/)
- [MCP Integration](https://google.github.io/adk-docs/mcp-integration/)

---

🎉 **Congratulations!** You've mastered agent tools and can now build sophisticated agents with custom capabilities!